In [ ]:
# ! pip install face_recognition

In [ ]:
import cv2
import face_recognition
import os
import numpy as np
import mediapipe as mp

folder_path = "/home/abir/Documents/FaceNet/data/"
classes = []
image_encodings = []
face_mesh = mp.solutions.face_mesh
ob_facemesh = face_mesh.FaceMesh()

def face_mesh_block(frame):
    RGB_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    height, width, _ = RGB_image.shape

    meshed_image = ob_facemesh.process(RGB_image)

    if meshed_image.multi_face_landmarks:  
        for facial_landmarks in meshed_image.multi_face_landmarks:
            for i in range(468):  
                point = facial_landmarks.landmark[i]
                x = int(point.x * width)
                y = int(point.y * height)

                cv2.circle(frame, (x, y), 1, (0, 255, 0), -1) 
    return frame

def prepare_encodings(folder_path):
    dir_content = os.listdir(folder_path)
    
    for file_name in dir_content:
        image_path = os.path.join(folder_path, file_name)
        image = cv2.imread(image_path)
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        encodings = face_recognition.face_encodings(rgb_image)

        if encodings:  
            image_encodings.append(encodings[0])
            classes.append(file_name[:-4]) 
    
    print(f"{len(image_encodings)} Image encoded from {folder_path}")


def compare_images(img):
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    face_locations = face_recognition.face_locations(rgb_img)
    face_encodings = face_recognition.face_encodings(rgb_img, face_locations)

    if not face_encodings:  
        return img, "No Face Detected"

    for encoding, (top, right, bottom, left) in zip(face_encodings, face_locations):
        matches = face_recognition.compare_faces(image_encodings, encoding)
        distances = face_recognition.face_distance(image_encodings, encoding)

        best_match_index = np.argmin(distances)  
        if matches[best_match_index]:  
            name = classes[best_match_index].upper()
        else:
            name = "NO MATCH"
        cv2.rectangle(img, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(img, name, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    return img, name
   


if __name__ == "__main__":
    prepare_encodings(folder_path)

    # IMADGE INFERENCE
    # test_image_path = "/home/abir/Documents/FaceNet/Dhoni-test.jpeg"
    # test_image_path = "/home/abir/Documents/FaceNet/Ghambhir-test.jpeg"
    test_image_path = "/home/abir/Documents/FaceNet/kat-test.jpeg"  #  Demostrating NO MATCH cases 


    test_image = cv2.imread(test_image_path)
    test_image = cv2.resize(test_image,(600,400))
    result_img, result_name = compare_images(test_image)

    mesh_result_img = face_mesh_block(frame=result_img)

    cv2.imshow("Face Recognition",mesh_result_img)
    cv2.waitKey(0)

    # REAL TIME INFERENCE       
 
    # cap = cv2.VideoCapture(0)       # UNCOMMENT TO TEST REAL TIME INFERENCE WITH WEB-CAM
    # while cap.isOpened():
    #     ret, frame = cap.read()
    #     if not ret:
    #         print("Check Camera Index")
    #         break
    #     frame = cv2.resize(frame,(600,400))
    #     result_img, result_name = compare_images(frame)
    #     mesh_result_img = face_mesh_block(frame=result_img)
    #     cv2.imshow("Face Recognition", mesh_result_img)

    #     if cv2.waitKey(1) & 0xFF == ord('q'):
    #         break

    # cap.release()
    # cv2.destroyAllWindows()


I0000 00:00:1742924170.160338   41389 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1742924170.180630   41462 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 550.120), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1742924170.183473   41457 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742924170.189993   41459 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


5 image encoded from /home/abir/Documents/FaceNet/data/


W0000 00:00:1742924174.924140   41458 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
